In [1]:
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments

# Replace with the path or name of your saved model
model_path = "./models/dalembert-ner-finetuned_ep5"
dataset_path = "./data/ck_ner_dataset_hg"
tokenizer_path = "./models/dalembert-ner-finetuned_tokenizer"

### Load our medieval french dataset (locally) & model and tokenizer
dataset = load_from_disk(dataset_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)

##### Tokenization still needs to be done.

In [ ]:
from hg_functions import get_label_mappings, tokenize_and_align_labels_for_ner, compute_metrics

label_list, label_to_id, id_to_label = get_label_mappings(dataset)

eval_dataset = dataset["validation"].map(
    lambda x: tokenize_and_align_labels_for_ner(x, tokenizer, label_to_id),
    batched=False
)
training_args = TrainingArguments(
    output_dir = "./results", 
    per_device_eval_batch_size = 8,
    do_eval = True,
    logging_dir = "./logs",
    eval_accumulation_steps = 2
)

trainer = Trainer(
    model = model,
    args = training_args,
    tokenizer = tokenizer,
    compute_metrics = lambda p: compute_metrics(p, id_to_label)
)


Map:   0%|          | 0/12700 [00:00<?, ? examples/s]

In [5]:
results = trainer.evaluate(eval_dataset)
print(results)

  0%|          | 0/1588 [00:00<?, ?it/s]

{'eval_loss': 0.018169540911912918, 'eval_precision': 0.9292389853137517, 'eval_recall': 0.9407716883755223, 'eval_f1': 0.9349697746840081, 'eval_accuracy': 0.9970777534928091, 'eval_runtime': 28.5147, 'eval_samples_per_second': 445.385, 'eval_steps_per_second': 55.691}
